In [ ]:
# criar o navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

#servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome()

In [ ]:
# abrir a página index (entrar no site da busca jurídica)
import os
caminho = os.getcwd()
arquivo = caminho + r"\index.html"

navegador.get(arquivo)

In [ ]:
from selenium.webdriver import ActionChains

#abrir lista de cidades
botao = navegador.find_element(By.CLASS_NAME, 'dropdown-menu')
ActionChains(navegador).move_to_element(botao).perform()

# Selecionando o Rio de Janeiro
navegador.find_element(By.XPATH, '/html/body/div/div/div/a[2]').click()

In [ ]:
# mudar para a nova aba
aba_original = navegador.window_handles[0]
nova_aba = navegador.window_handles[1]

navegador.switch_to.window(nova_aba)

# preencher o formulário com os dados de busca
navegador.find_element(By.ID, 'nome').send_keys('Gabriel')
navegador.find_element(By.ID, 'advogado').send_keys('Jesus Cristo')
navegador.find_element(By.ID, 'numero').send_keys('77')

# clicar em pesquisar
navegador.find_element(By.CLASS_NAME, 'registerbtn').click()

In [ ]:
# confirmar a pesquisa
alerta = navegador.switch_to.alert
alerta.accept()

In [ ]:
# esperar o resultado da pesquisa e agir de acordo com o resultado
from cgitb import text
import time 

while True:
  try:
    alerta = navegador.switch_to.alert
    break
  except:
    time.sleep(1) 
texto_alerta = alerta.text

if 'Processo encontrado com sucesso' in texto_alerta:
  alerta.accept()
  print('Processo foi encontrado!')
else:
  print('Processo não encontrado!')
  alerta.accept()